<a href="https://colab.research.google.com/github/dolya-julia/neural_network_1/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam

In [ ]:
# Каталог с данными для обучения
train_dir = "/content/drive/MyDrive/dataset/train/"
# Каталог с данными для проверки
val_dir = '/content/drive/MyDrive/dataset/val/'
# Каталог с данными для тестирования
test_dir = '/content/drive/MyDrive/dataset/test/'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
epochs = 30
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 9716
# Количество изображений для проверки
nb_validation_samples = 3003
# Количество изображений для тестирования
nb_test_samples = 3033

In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg16_net.trainable = False

In [ ]:
vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Flatten())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 16812610 (64.14 MB)
Trainable params: 2097

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-5),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 9716 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3003 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3033 images belonging to 2 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

<ipython-input-18-0b2ca182ead4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
151/151 [==============================] - 4921s 33s/step - loss: 0.6357 - accuracy: 0.6418 - val_loss: 0.5667 - val_accuracy: 0.7235
Epoch 2/10
151/151 [==============================] - 352s 2s/step - loss: 0.5138 - accuracy: 0.7280 - val_loss: 0.5457 - val_accuracy: 0.7313
Epoch 3/10
151/151 [==============================] - 356s 2s/step - loss: 0.4781 - accuracy: 0.7611 - val_loss: 0.5508 - val_accuracy: 0.7160
Epoch 4/10
151/151 [==============================] - 324s 2s/step - loss: 0.4580 - accuracy: 0.7725 - val_loss: 0.5392 - val_accuracy: 0.7262
Epoch 5/10
151/151 [==============================] - 318s 2s/step - loss: 0.4518 - accuracy: 0.7748 - val_loss: 0.5085 - val_accuracy: 0.7432
Epoch 6/10
151/151 [==============================] - 343s 2s/step - loss: 0.4392 - accuracy: 0.7854 - val_loss: 0.5167 - val_accuracy: 0.7408
Epoch 7/10
151/151 [==============================] - 310s 2s/step - loss: 0.4344 - accuracy: 0.7834 - val_loss: 0.5262 - val_accuracy: 0.72

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

<ipython-input-20-4f9ca0ff4799>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Аккуратность на тестовых данных: 77.76%


In [ ]:
model_json = model.to_json()
json_file = open("fine_tuning_cifar11_model.json","w")
json_file.write(model_json)
json_file.close()
model.save_weights("fine_tuning_cifar11_model.h5")

In [ ]:
!ls

drive  fine_tuning_cifar11_model.h5  fine_tuning_cifar11_model.json  sample_data


In [ ]:
from google.colab import files
files.download("fine_tuning_cifar11_model.json")
files.download("fine_tuning_cifar11_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("fine_tuning_cifar11_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>